In [11]:
import pandas as pd
dtypes = {
    'MAPID': 'object',  # UUID
    'CTV3_CONCEPTID': 'object',  # VARCHAR(5)
    'CTV3_TERMID': 'object',  # VARCHAR(5)
    'CTV3_TERMTYPE': 'object',  # CHAR(1)
    'SCT_CONCEPTID': 'object',  # VARCHAR(18)
    'SCT_DESCRIPTIONID': 'object',  # VARCHAR(18)
    'MAPSTATUS': 'int8',  # TINYINT
    'EFFECTIVEDATE': 'int32',  # YYYYMMDD as INT
    'IS_ASSURED': 'int8'  # TINYINT
}
# Load the data
file_path = 'ctv3sctmap2_uk_20200401000001.txt'
df = pd.read_csv(file_path,low_memory=True, delimiter='\t', dtype=dtypes)
df['EFFECTIVEDATE'] = pd.to_datetime(df['EFFECTIVEDATE'], format='%Y%m%d')


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 861837 entries, 0 to 861836
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   MAPID              861837 non-null  object        
 1   CTV3_CONCEPTID     861837 non-null  object        
 2   CTV3_TERMID        861837 non-null  object        
 3   CTV3_TERMTYPE      679684 non-null  object        
 4   SCT_CONCEPTID      861837 non-null  object        
 5   SCT_DESCRIPTIONID  615077 non-null  object        
 6   MAPSTATUS          861837 non-null  int8          
 7   EFFECTIVEDATE      861837 non-null  datetime64[ns]
 8   IS_ASSURED         861837 non-null  int8          
dtypes: datetime64[ns](1), int8(2), object(6)
memory usage: 47.7+ MB


In [13]:
active_preferred_mappings = df[(df['MAPSTATUS'] == 1) & (df['CTV3_TERMTYPE'] == 'P')]


In [14]:
active_preferred_mappings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 421878 entries, 13 to 861836
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   MAPID              421878 non-null  object        
 1   CTV3_CONCEPTID     421878 non-null  object        
 2   CTV3_TERMID        421878 non-null  object        
 3   CTV3_TERMTYPE      421878 non-null  object        
 4   SCT_CONCEPTID      421878 non-null  object        
 5   SCT_DESCRIPTIONID  262143 non-null  object        
 6   MAPSTATUS          421878 non-null  int8          
 7   EFFECTIVEDATE      421878 non-null  datetime64[ns]
 8   IS_ASSURED         421878 non-null  int8          
dtypes: datetime64[ns](1), int8(2), object(6)
memory usage: 26.6+ MB


In [15]:
ctv3_to_snomed_counts = active_preferred_mappings.groupby('CTV3_CONCEPTID')['SCT_CONCEPTID'].nunique()
snomed_to_ctv3_counts = active_preferred_mappings.groupby('SCT_CONCEPTID')['CTV3_CONCEPTID'].nunique()


In [28]:
ctv3_to_snomed_counts.reset_index(name='Unique SNOMED Concepts Count').describe()

,Unique SNOMED Concepts Count
count,276841.000000
mean,1.115348
std,0.348359
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,6.000000


In [67]:
snomed_to_ctv3_df = snomed_to_ctv3_counts.reset_index(name='Unique CTV3 Concepts Count')
snomed_to_ctv3_df.drop(index=182680, inplace=True)
snomed_to_ctv3_df[snomed_to_ctv3_df['Unique CTV3 Concepts Count'] > 1].sort_values('Unique CTV3 Concepts Count', ascending=False).head(30)


,SCT_CONCEPTID,Unique CTV3 Concepts Count
159913,52684005,209
41217,214640008,193
132676,34552002,128
153142,449171000000101,122
146338,418611001,102
42683,217176002,101
140922,399269003,96
6986,116360008,96
146206,418019003,93
9843,127349007,90


In [33]:
def get_ctv3_codes_for_snomed(snomed_code):
    linked_ctv3_codes = active_preferred_mappings[active_preferred_mappings['SCT_CONCEPTID'] == snomed_code]
    return linked_ctv3_codes[['CTV3_CONCEPTID', 'CTV3_TERMID', 'SCT_DESCRIPTIONID']]

# Example usage of the function
snomed_code_example = '229059009'  # Replace with an actual SNOMED code
linked_ctv3_codes_example = get_ctv3_codes_for_snomed(snomed_code_example)
print(f"CTV3 codes linked to SNOMED code {snomed_code_example}:\n", linked_ctv3_codes_example)

CTV3 codes linked to SNOMED code 229059009:
        CTV3_CONCEPTID CTV3_TERMID SCT_DESCRIPTIONID
202567          9EY..       Ya2z1        2668995012
357450          Ub1Fg       YMDZy        2668995012
357452          Ub1Fg       YMDZy         343391017
357453          Ub1Fg       YMDZy         343391017


In [68]:
description_dtypes = {
    'id': 'object',
    'effectiveTime': 'int32',
    'active': 'int8',
    'moduleId': 'object',
    'conceptId': 'object',
    'languageCode': 'object',
    'typeId': 'object',
    'term': 'object',
    'caseSignificanceId': 'object'
}
# Load the description data
description_file_path = 'other/sct2_Description_MONOSnapshot-en_GB_20240703.txt'
df_description = pd.read_csv(description_file_path, delimiter='\t', dtype=description_dtypes)

# Filter the active descriptions
active_descriptions = df_description[df_description['active'] == 1]

# Select only the relevant columns for the merge
active_descriptions = active_descriptions[['id', 'term']]

# Merge the mappings with the descriptions to get the term
merged_df = pd.merge(active_preferred_mappings, active_descriptions, left_on='SCT_DESCRIPTIONID', right_on='id', how='left')

# Select relevant columns for the reference DataFrame
reference_df = merged_df[['SCT_CONCEPTID', 'CTV3_CONCEPTID', 'CTV3_TERMID', 'term']]

# Save the reference DataFrame for future lookups
reference_df.to_csv('reference_dataframe.csv', index=False)

In [78]:
def get_ctv3_codes_for_snomed(snomed_code, reference_df):
    linked_ctv3_codes = reference_df[reference_df['SCT_CONCEPTID'] == snomed_code]
    return linked_ctv3_codes

# Load the reference DataFrame (to simulate future lookups)
reference_df = pd.read_csv('reference_dataframe.csv')

# Example usage of the function
snomed_code_example = '80394007'  # Replace with an actual SNOMED code
linked_ctv3_codes_example = get_ctv3_codes_for_snomed(snomed_code_example, reference_df)
print(f"CTV3 codes and descriptions linked to SNOMED code {snomed_code_example}:\n", linked_ctv3_codes_example)

CTV3 codes and descriptions linked to SNOMED code 80394007:
        SCT_CONCEPTID CTV3_CONCEPTID CTV3_TERMID            term
72661       80394007          Ryu8A       YMAY2   Hyperglycemia
72663       80394007          Ryu8A       YMAY2   Hyperglycemia
72664       80394007          Ryu8A       YMAY2  Hyperglycaemia
72665       80394007          Ryu8A       YMAY2  Hyperglycaemia
118986      80394007          X40J2       Y41Ov  Hyperglycaemia
225331      80394007          XaFxf       YM13z   Hyperglycemia
225332      80394007          XaFxf       YM13z  Hyperglycaemia
225333      80394007          XaFxf       YM13z  Hyperglycaemia


In [77]:
import pandas as pd
from IPython.display import display

# Define the correct data types for the mapping file
mapping_dtypes = {
    'MAPID': 'object',  # UUID
    'CTV3_CONCEPTID': 'object',  # VARCHAR(5)
    'CTV3_TERMID': 'object',  # VARCHAR(5)
    'CTV3_TERMTYPE': 'object',  # CHAR(1)
    'SCT_CONCEPTID': 'object',  # VARCHAR(18)
    'SCT_DESCRIPTIONID': 'object',  # VARCHAR(18)
    'MAPSTATUS': 'int8',  # TINYINT
    'EFFECTIVEDATE': 'int32',  # YYYYMMDD as INT
    'IS_ASSURED': 'int8'  # TINYINT
}

# Load the mapping data
mapping_file_path = 'ctv3sctmap2_uk_20200401000001.txt'
df_mapping = pd.read_csv(mapping_file_path, delimiter='\t', dtype=mapping_dtypes)

# Convert EFFECTIVEDATE to datetime
df_mapping['EFFECTIVEDATE'] = pd.to_datetime(df_mapping['EFFECTIVEDATE'], format='%Y%m%d')

# Filter the active and preferred mappings
active_preferred_mappings = df_mapping[(df_mapping['MAPSTATUS'] == 1) & (df_mapping['CTV3_TERMTYPE'] == 'P')]

# Define the correct data types for the description file
description_dtypes = {
    'id': 'object',
    'effectiveTime': 'int32',
    'active': 'int8',
    'moduleId': 'object',
    'conceptId': 'object',
    'languageCode': 'object',
    'typeId': 'object',
    'term': 'object',
    'caseSignificanceId': 'object'
}

# Load the description data
description_file_path = 'other/sct2_Description_MONOSnapshot-en_GB_20240703.txt'
df_description = pd.read_csv(description_file_path, delimiter='\t', dtype=description_dtypes)

# Filter the active descriptions
active_descriptions = df_description[df_description['active'] == 1]

# Select only the relevant columns for the merge
active_descriptions = active_descriptions[['id', 'term']]

# Merge the mappings with the descriptions to get the term
merged_df = pd.merge(active_preferred_mappings, active_descriptions, left_on='SCT_DESCRIPTIONID', right_on='id', how='left')

# Select relevant columns for the reference DataFrame
reference_df = merged_df[['SCT_CONCEPTID', 'CTV3_CONCEPTID', 'CTV3_TERMID', 'term', 'EFFECTIVEDATE']]

# Sort by EFFECTIVEDATE in descending order and drop duplicates, keeping the most recent
reference_df = reference_df.sort_values(by='EFFECTIVEDATE', ascending=False).drop_duplicates(subset=['CTV3_CONCEPTID', 'CTV3_TERMID'])

# Drop the EFFECTIVEDATE column as it's no longer needed
reference_df = reference_df.drop(columns=['EFFECTIVEDATE'])

# Save the reference DataFrame for future lookups
reference_df.to_csv('reference_dataframe_v2.csv', index=False)

# Load the reference DataFrame (to simulate future lookups)
reference_df2 = pd.read_csv('reference_dataframe_v2.csv')




/tmp/ipykernel_90622/294639667.py:66: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  reference_df2 = pd.read_csv('reference_dataframe_v2.csv')


In [93]:
code = '13645005'
# Example usage of the function
snomed_code_example = code  # Replace with an actual SNOMED code
linked_ctv3_codes_example = get_ctv3_codes_for_snomed(snomed_code_example, reference_df)
print(f"CTV3 codes and descriptions linked to SNOMED code {snomed_code_example}:\n", linked_ctv3_codes_example)
# Example usage of the function
snomed_code_example = code  # Replace with an actual SNOMED code
linked_ctv3_codes_example = get_ctv3_codes_for_snomed(snomed_code_example, reference_df2)
print(f"CTV3 codes and descriptions linked to SNOMED code {snomed_code_example}:\n", linked_ctv3_codes_example)

CTV3 codes and descriptions linked to SNOMED code 13645005:
        SCT_CONCEPTID CTV3_CONCEPTID CTV3_TERMID  \
53138       13645005          H3...       Y106U   
53154       13645005          H312z       Y106r   
53245       13645005          H3y..       Y106g   
53246       13645005          H3y..       Y106g   
53248       13645005          H3z..       Y106h   
53249       13645005          H3z..       Y106h   
53643       13645005          Hyu31       YMAJK   
53644       13645005          Hyu31       YMAJK   
231374      13645005          XaK8Q       YanLZ   

                                              term  
53138             Chronic obstructive lung disease  
53154             Chronic obstructive lung disease  
53245             Chronic obstructive lung disease  
53246           Chronic obstructive airway disease  
53248   COAD - Chronic obstructive airways disease  
53249           Chronic obstructive airway disease  
53643        Chronic obstructive pulmonary disease  
5364